In [2]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import time
import progressbar
from scipy import stats

In [3]:
filenames = ['../../data/bookcorssing_r05.csv',
             '../../data/bookcorssing_r10.csv',
             '../../data/audioscrobbler.csv',  
             '../../data/amazonbook.csv',
             '../../data/amazoncd.csv',  
             '../../data/amazonelectronic.csv',
             '../../data/amazonmovie.csv',
             '../../data/facebooklike_msg.csv',
             '../../data/algorithm_r05.csv',
             '../../data/artificialintelligence_r05.csv',
             '../../data/programminglanguage_r05.csv'
            ]
threshs = [185, 185, 1250, 450, 160, 60, 150, 10, 100, 120, 105]
blocks = [0,0,0,0,0,0,0,0,0,0,0]

In [4]:
def get_valid_data(filename,block):
    '''    first ,kick out invalid data -- rating time less than thresh   '''
    #allData=pd.read_csv('BX-Book-Ratings.csv')
    #print(allData.columns )
    allData=dict()
    pos = 0
    with open(filename,'r') as f:
            # filter the first line
        f.readline()  
        lines=f.readlines()
            # traverse and filter 
        for idx,line in enumerate(lines):
            if block == 1:
                dataInString=line.strip().split(';')  # list type
            else:
                dataInString=line.strip().split(',')  # list type
            for index,data in enumerate(dataInString):  # userName ISBN score
                dataInString[index]=data.replace('\"','')
            if not allData.get(dataInString[0]) :   # create the user's list if he does not exist
                allData[dataInString[0]]=[]
            allData[dataInString[0]].append(float(dataInString[2]))
    #        miu.append( float(dataInString[2]) )
    return allData
    '''calculate miu1 and miu2 using allData but predict only with valid users'''

In [5]:
def get_valid_user(allData, thresh):
    # %% cell1
    ''' pick out valid user '''
    #num=0
    miu=[]    # to store all data
    rateFreq=dict() # key is rate and value is its frequency  rateFreq[userName][score]=frequency
    for key in allData.keys():      # userName
        if len(allData[key])> thresh:
    #        num+=1
                # create the dict for each valid user
            if not rateFreq.get( key ):
                rateFreq[ key ]= dict()     
                # create the dict entry for each score
            for score in allData[key]:
                if not rateFreq[key].get(score):
                    rateFreq[ key ][ score ]=0
                rateFreq[ key ][ score ]+=1
                miu.append(score)
    return rateFreq, miu

In [6]:
def cal_para_multiple(rateFreq, miu, write_multiple):
    # %% cell2
    '''calculate common para'''
    miu1=np.mean(miu);  theta1=np.std(miu)
    # four direction for miu2 and theta2 
    # deltamiu=[0.5,-0.5]; theta=[2,1,-1,-2]
    cp = [(0,1), (0,-1), (1,-1), (-1,1)]

    write=open(write_multiple,'w');

    N=8*len(rateFreq)
    p = progressbar.ProgressBar(maxval=N)
    p.start() 
    progress=0

    userWeight=[None]*8   # list of 4 ele
    for i in range(2):
        for j in range(4):
            deltamiu = cp[j][0]
            theta = cp[j][1]

            miu2=miu1+deltamiu
            theta2=theta1+theta

            userWeight[i*4+j]=dict()
                # rateFreq[userName][score]=frequency
                # userScore[score]=frequency
            '''test'''
            print("************************************",file=write)
            print('i={0},j={1}'.format(i,j),file=write )
            sumMse_all = 0.0
            for user,userScore in rateFreq.items(): 
                    # cal MSE and choose the best weight
                bestWeight=0
                minMse=float('inf');
                '''对每个user 遍历一遍weight 把每个数据点都相加'''
                for weight in np.arange(0, 1.1 ,0.1): # arange exclude endPoint while linspace includes defaultly  
                    sumMse=0
                        # userScore[score]=frequency
                    '''对于每个user，他对每个score的mse要累计起来'''
                    '''这里需要check下data'''
                    for score,freq in userScore.items():    
                        sumMse+=(( weight*stats.norm(miu1, theta1).cdf(score)+ (1-weight)*stats.norm(miu2,theta2).cdf(score)- freq ) )**2
                    
                    '''test'''
                    print('user:{0},weight:{1},mse:{2}'.format(user,weight,sumMse),file=write )
                   
                    if sumMse < minMse:
                        minMse=sumMse
                        bestWeight=weight
                sumMse_all += minMse
                '''test'''
                print('user:{0},minMse:{1}'.format(user,minMse),file=write )
                userWeight[i*4+j][user]=bestWeight;   # change here   
                
                progress+=1
                p.update(progress)
            print(sumMse_all/len(rateFreq.keys()))
            print(sumMse_all/len(rateFreq.keys()),file=write )

    
    print(userWeight,file=write)
    write.close()
    p.finish()

In [7]:
def cal_para_single(rateFreq, miu, write_single):
    # %% cell2
    '''calculate common para'''
    miu1=np.mean(miu);  theta1=np.std(miu)
    # four direction for miu2 and theta2 
    # deltamiu=[0.5,-0.5]; theta=[2,1,-1,-2]
    cp = [(0,1), (0,-1), (1,-1), (-1,1)]

    write=open(write_single,'w');


    N=8*len(rateFreq)
    p = progressbar.ProgressBar(maxval=N)
    p.start() 
    progress=0

    userWeight=[None]*8   # list of 4 ele
    for i in range(2):
        for j in range(1):
            
            userWeight[i*4+j]=dict()
                # rateFreq[userName][score]=frequency
                # userScore[score]=frequency
            '''test'''
            print("************************************",file=write)
            print('i={0},j={1}'.format(i,j),file=write )
            sumMse_all = 0.0
            for user,userScore in rateFreq.items(): 
                    # cal MSE and choose the best weight
                bestWeight=0
                minMse=float('inf');
                '''对每个user 遍历一遍weight 把每个数据点都相加'''
                for weight in np.arange(0, 1.1 ,0.1): # arange exclude endPoint while linspace includes defaultly  
                    sumMse=0
                        # userScore[score]=frequency
                    '''对于每个user，他对每个score的mse要累计起来'''
                    '''这里需要check下data'''
                    for score,freq in userScore.items():    
                        sumMse+=((weight*stats.norm(miu1, theta1).cdf(score)- freq ))**2
                    
                    '''test'''
                    print('user:{0},weight:{1},mse:{2}'.format(user,weight,sumMse),file=write )
                   
                    if sumMse < minMse:
                        minMse=sumMse
                        bestWeight=weight
                sumMse_all += minMse
                '''test'''
                print('user:{0},minMse:{1}'.format(user,minMse),file=write )
                userWeight[i*4+j][user]=bestWeight;   # change here   
                
                progress+=1
                p.update(progress)
            print(sumMse_all/len(rateFreq.keys()),)
            print(sumMse_all/len(rateFreq.keys()),file=write)

    
    print(userWeight,file=write)
    write.close()
    p.finish()

In [9]:
if __name__=="__main__":
    

../../data/amazonelectronic_single_1.txt ../../data/amazonelectronic_multiple_1.txt


# DATA Regression User

## I=0 bookcorssing_r05

In [24]:
i = 0
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/bookcorssing_r05_single_1.txt ../../data/bookcorssing_r05_multiple_1.txt


355234.33939775807


100% |########################################################################|


355234.33939775807


355213.4836909893


...

355083.9462801417


355213.4836909893


inf


inf


355083.9462801417


## I=1 bookcorssing_r10.csv

In [25]:
i = 1
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/bookcorssing_r10_single_1.txt ../../data/bookcorssing_r10_multiple_1.txt


212241.76320303077


100% |########################################################################|


212241.76320303077


212230.20506098188


...

212150.88157271734


212230.20506098188


212208.56890978332


212241.3838211812


212150.88157271734


## I=2 audioscrobbler

In [26]:
i = 2
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/audioscrobbler_single_1.txt ../../data/audioscrobbler_multiple_1.txt


1013096.972273024


100% |########################################################################|


1013096.972273024


1013004.5880616489


...

1012500.6459892872


1013004.5880616489


1013094.2184864104


1013096.972273024


1012500.6459892872


## I=3 amazonbook

In [27]:
i = 3
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonbook_single_1.txt ../../data/amazonbook_multiple_1.txt


2854000.8507108646


2854000.8507108646


2853974.575037313


inf


...

2853724.5295482827


2853974.575037313


inf


inf


2853724.5295482827


## I=4 amazoncd

In [28]:
i = 4
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazoncd_single_1.txt ../../data/amazoncd_multiple_1.txt


159947.8862267593


100% |########################################################################|


159947.8862267593


159938.7330258334


...

159835.03615094768


159938.7330258334


inf


inf


159835.03615094768


## I=5 amazonelectronic

In [29]:
i = 5
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonelectronic_single_1.txt ../../data/amazonelectronic_multiple_1.txt


5582.597239184795


100% |########################################################################|


5582.597239184795


5581.55314619139


...

5556.626213107219


5581.55314619139


5572.816914775412


5582.597239184795


5556.626213107219


## I=6 amazonmovie

In [30]:
i = 6
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/amazonmovie_single_1.txt ../../data/amazonmovie_multiple_1.txt


89724.90925648874


100% |########################################################################|


89724.90925648874


89716.08464820682


...

89624.46624907461


89716.08464820682


89659.5235727073


89724.90925648874


89624.46624907461


## I=7 facebooklike_msg

In [31]:
i = 7
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/facebooklike_msg_single_1.txt ../../data/facebooklike_msg_multiple_1.txt


92.83415895623406


100% |########################################################################|


92.83415895623406


92.52968727950457


...

91.44231228826256


92.52968727950457


92.8300091689989


92.83415895623406


91.44231228826256


## I=8 algorithm

In [32]:
i = 8
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/algorithm_r05_single_1.txt ../../data/algorithm_r05_multiple_1.txt


8145.460509611751


100% |########################################################################|


8145.460509611751


8134.80448575585


...

8079.238764326332


8134.80448575585


inf


inf


8079.238764326332


## I=9 ai

In [33]:
i = 9
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/artificialintelligence_r05_single_1.txt ../../data/artificialintelligence_r05_multiple_1.txt


22305.285195757882


100% |########################################################################|


22305.285195757882


22290.061814038992


...

22214.9527628158


22290.061814038992


inf


inf


22214.9527628158


## I=10 PL

In [34]:
i = 10
c = filenames[i]
c = c.replace('ratings_', '')
s_out = c.replace('.csv','')+'_single_1.txt'
m_out = c.replace('.csv','')+'_multiple_1.txt'
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filenames[i], blocks[i]), threshs[i])
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

../../data/programminglanguage_r05_single_1.txt ../../data/programminglanguage_r05_multiple_1.txt


19143.68701702681


100% |########################################################################|


19143.68701702681


19130.259184389375


...

19066.379811170274


19130.259184389375


inf


inf


19066.379811170274


In [ ]:
c = "libimseti_f_m.csv"
s_out = c.replace('.csv','')+'_single_u.txt'
m_out = c.replace('.csv','')+'_multiple_u.txt'
filename = "../data/libimseti_f_m.csv"
block = 0
thresh = 100
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filename, block), thresh)
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out) 

In [8]:
c = "libimseti_m_f.csv"
s_out = c.replace('.csv','')+'_single_u.txt'
m_out = c.replace('.csv','')+'_multiple_u.txt'
filename = "../data/libimseti_m_f.csv"
block = 0
thresh = 100 
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filename, block), thresh)
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

libimseti_m_f_single_u.txt libimseti_m_f_multiple_u.txt


55145.696662021866


55145.696662021866


55140.10940047691


55141.290124398205


...

55093.48075222508


55140.10940047691


55141.290124398205


55145.62304356516


55093.48075222508


## 4-17

In [10]:
c = "libimseti_m_f_r05.csv"
s_out = c.replace('.csv','')+'_single_r05_u.txt'
m_out = c.replace('.csv','')+'_multiple_r05_u.txt'
filename = "../data/libimseti_m_f.csv"
block = 0
thresh = 200 
print(s_out, m_out)
a_dict, a_miu = get_valid_user(get_valid_data(filename, block), thresh)
cal_para_single(a_dict, a_miu, s_out)
cal_para_multiple(a_dict, a_miu, m_out)

libimseti_m_f_r05_single_r05_u.txt libimseti_m_f_r05_multiple_r05_u.txt


140467.498857415


KeyboardInterrupt: 